In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [20]:
class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(
            in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_planes, eps=0.001)
        self.relu = nn.ReLU(inplace=True)
        

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

### Calculate Paramters

In [21]:
model = nn.Conv2d(12, 12, kernel_size=(3, 3), stride=1, padding=1, bias=True)
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"params : {params}")
print(f"calculation : {((3 * 3) * 12 + 1) * 12}")

print("\n")

model = BasicConv2d(12, 12, 3, 1, 1)
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"params : {params}")
print(f"calculation : {((3 * 3) * 12 ) * 12 + 12 * 2}")

params : 1308
calculation : 1308


params : 1320
calculation : 1320


In [23]:
from torchsummary import summary
summary(model, (12, 32, 32), batch_size=1, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 12, 32, 32]           1,296
       BatchNorm2d-2            [1, 12, 32, 32]              24
              ReLU-3            [1, 12, 32, 32]               0
Total params: 1,320
Trainable params: 1,320
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.28
Params size (MB): 0.01
Estimated Total Size (MB): 0.33
----------------------------------------------------------------


## Inception

![](assets/inception.png)

#### examples

In [4]:
class InceptionBlock(nn.Module):
    def __init__(self, in_planes: int = 192):
        super().__init__()
        self.branch1 = BasicConv2d(in_planes, 64, kernel_size=(1, 1), stride=1, padding=0)
        self.branch2 = nn.Sequential(
            BasicConv2d(in_planes, 96, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(96, 128, kernel_size=(3, 3), stride=1, padding=1)
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_planes, 16, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(16, 32, kernel_size=(5, 5), stride=1, padding=2)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(3, 3), stride=1, padding=1),
            BasicConv2d(in_planes, 32, kernel_size=(1, 1), stride=1, padding=0)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        output = torch.cat((x1, x2, x3, x4), dim=1)
        return output 


In [5]:
model = InceptionBlock(192)

In [6]:
batch_size = 4
features = torch.randn(batch_size, 192, 28, 28)
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 192, 28, 28])
Output shape : torch.Size([4, 256, 28, 28])


#### practice 

1. V1
    * input_shape = (batch_size, 64, 28, 28)
    * output_shape = (batch_size, 128, 28, 28)
    * 4 branches, must have kernel (1, 1), (3, 3), (5, 5), (7, 7)


2. V3
    * input_shape = (batch_size, 64, 28, 28)
    * output_shape = (batch_size, 128, 28, 28)
    * 4 branches, must have kernel (1, 1), (3, 1), (1, 3), (5, 1), (1, 5)

In [7]:
features = torch.randn(batch_size, 64, 28, 28)

In [8]:
class MyInceptionBlock(nn.Module):
    def __init__(self, in_planes: int = 64):
        super().__init__()
        self.branch1 = BasicConv2d(in_planes, 32, kernel_size=(1, 1), stride=1, padding=0)
        self.branch2 = nn.Sequential(
            BasicConv2d(in_planes, 16, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(16, 32, kernel_size=(3, 3), stride=1, padding=1)
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_planes, 16, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(16, 32, kernel_size=(5, 5), stride=1, padding=2)
        )
        self.branch4 = nn.Sequential(
            BasicConv2d(in_planes, 16, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(16, 32, kernel_size=(7, 7), stride=1, padding=3)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        output = torch.cat((x1, x2, x3, x4), dim=1)
        return output

In [9]:
model = MyInceptionBlock()
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 64, 28, 28])
Output shape : torch.Size([4, 128, 28, 28])


In [10]:
class InceptionBlockV3(nn.Module):
    def __init__(self, in_planes: int = 64):
        super().__init__()
        self.branch1 = BasicConv2d(in_planes, 28, kernel_size=(1, 1), stride=1, padding=0)
        self.branch2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(3, 3), stride=1, padding=1),
            BasicConv2d(in_planes, 20, kernel_size=(1, 1), stride=1, padding=0)
        )
        self.branch3_1 = nn.Sequential(
            BasicConv2d(in_planes, 16, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(16, 20, kernel_size=(3, 1), stride=1, padding=(1, 0))
        )
        self.branch3_2 = nn.Sequential(
            BasicConv2d(in_planes, 16, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(16, 20, kernel_size=(1, 3), stride=1, padding=(0, 1))
        )
        self.branch4_0 = nn.Sequential(
            BasicConv2d(in_planes, 16, kernel_size=(1, 1), stride=1, padding=0),
            BasicConv2d(16, 32, kernel_size=(5, 5), stride=1, padding=2)
        )
        self.branch4_1 = nn.Sequential(
            BasicConv2d(16, 32, kernel_size=(5, 1), stride=1, padding=(2, 0))
        )
        self.branch4_2 = nn.Sequential(
            BasicConv2d(16, 32, kernel_size=(1, 5), stride=1, padding=(0, 2))
        )

        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3_1 = self.branch3_1(x)
        x3_2 = self.branch3_2(x)
        x4_0 = self.branch4_0(x)
        x4_1 = self.branch4_1(x4_0)
        x4_2 = self.branch4_2(x4_0)
        output = torch.cat((x1, x2, x3_1, x3_2, x4_1, x4_2), dim=1)
        return output 

In [11]:
model = MyInceptionBlock()
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 64, 28, 28])
Output shape : torch.Size([4, 128, 28, 28])


---

## ResNet - Residual Block

![](assets/residual.png)

![](assets/residual_differentiate.png)

#### examples

In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, in_planes: int, out_planes: int, stride: int = 1):
        super().__init__()
        self.conv1 = BasicConv2d(in_planes, out_planes, (3, 3), stride, 1)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=(3, 3), stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)
        if (in_planes != out_planes) or (stride != 1):
            self.map = nn.Conv2d(in_planes, out_planes, kernel_size=(1, 1), stride=stride, padding=0, bias=False)
        else:
            self.map = nn.Identity()
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.map(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.bn(x)
        x += identity
        x = self.relu(x)
        return x 

In [13]:
model = ResidualBlock(64, 64, 1)

In [14]:
batch_size = 4
features = torch.randn(batch_size, 64, 32, 32)
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 64, 32, 32])
Output shape : torch.Size([4, 64, 32, 32])


#### v2

![](assets/r1_r2.png)

#### Residual Branch
* Remove Relu activiation function : Avoid monotone increasing.
#### Identity Branch
* Remove BN, which could change the distribution.

In [15]:
class ResidualBlockV2(nn.Module):
    def __init__(self, in_planes: int, out_planes: int, stride: int = 1):
        super().__init__()
        squeeze = out_planes // 4
        self.conv1 = BasicConv2d(in_planes, squeeze, (1, 1), 1, 0) ## squeeze
        self.conv2 = BasicConv2d(squeeze, squeeze, (3, 3), stride=stride, padding=1) ## squeeze
        self.conv3 = nn.Conv2d(squeeze, out_planes, (1, 1), 1, 0) ## expand
        if (in_planes != out_planes) or (stride != 1):
            self.map = nn.Conv2d(in_planes, out_planes, kernel_size=(1, 1), stride=stride, padding=0, bias=False)
        else:
            self.map = nn.Identity()
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.map(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x += identity
        return x 

In [16]:
model = ResidualBlockV2(64, 64, 1)

In [17]:
batch_size = 4
features = torch.randn(batch_size, 64, 32, 32)
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 64, 32, 32])
Output shape : torch.Size([4, 64, 32, 32])


#### practice
* combine Inception with Residual

![](assets/inceptionv4.png)

In [18]:
batch_size = 4
features = torch.randn(batch_size, 32, 32, 32)

In [19]:
class InceptionResBlock(nn.Module):
    def __init__(self, in_planes: int, out_planes: int, stride: int = 1):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.branch1 = BasicConv2d(in_planes, 32, (1, 1), 1, 0)
        self.branch2 = nn.Sequential(
            BasicConv2d(in_planes, 32, (1, 1), 1),
            BasicConv2d(32, 32, (3, 3), 1, 1)
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_planes, 32, (1, 1), 1),
            BasicConv2d(32, 48, (3, 3), 1, 1),
            BasicConv2d(48, 64, (3, 3), 1, 1)
        )
        self.branch4 = BasicConv2d(128, 384, (1, 1), stride, 0)

        if (in_planes != out_planes) or (stride != 1):
            self.map = nn.Conv2d(in_planes, out_planes, kernel_size=(1, 1), stride=stride, padding=0, bias=False)
        else:
            self.map = nn.Identity()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.relu(x)
        identity = self.map(x)

        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x1_3 = torch.cat((x1, x2, x3), dim=1)
        x4 = self.branch4(x1_3)

        x = identity + x4

        output = self.relu(x)
        return output 

In [20]:
model = InceptionResBlock(32, 384)
output = model(features)
print(f"Input shape : {features.shape}\nOutput shape : {output.shape}")

Input shape : torch.Size([4, 32, 32, 32])
Output shape : torch.Size([4, 384, 32, 32])
